# CSP Strategy — Slim Notebook

Imports from `csp/` package. All logic lives in modules, tested with 216 tests.

**Structure:**
1. Imports & Config
2. Connect & Initialize
3. Portfolio Status
4. Scan Diagnostics
5. Run Strategy
6. Utilities

In [1]:
# Imports
from dotenv import load_dotenv
load_dotenv(override=True)

from csp.config import StrategyConfig
from csp.clients import AlpacaClientManager
from csp.data.vix import VixDataFetcher
from csp.data.greeks import GreeksCalculator
from csp.data.manager import DataManager
from csp.signals.scanner import StrategyScanner
from csp.signals.diagnostics import print_scan_diagnostics
from csp.trading.execution import ExecutionEngine
from csp.trading.risk import RiskManager
from csp.trading.metadata import StrategyMetadataStore
from csp.trading.loop import TradingLoop
from csp.trading.portfolio import print_portfolio_status
from csp.trading.liquidation import liquidate_all_holdings

print("Imports OK")

Imports OK


In [2]:
# Configuration — edit overrides here
config = StrategyConfig(
    # Uncomment/edit to override defaults:
    # ticker_universe=["AAPL", "MSFT", "GOOG"],
    # starting_cash=100_000,
    # num_tickers=5,
    paper_trading=True,
    entry_order_type="market",  # "market" (reliable with delayed data) or "stepped"
)

print(f"Config: {len(config.ticker_universe)} symbols, cash=${config.starting_cash:,}")
print(f"Paper trading: {config.paper_trading}")
print(f"Entry: type={config.entry_order_type}, start={config.entry_start_price}, "
      f"interval={config.entry_step_interval}s, steps={config.entry_max_steps}")

Config: 501 symbols, cash=$1,000,000
Paper trading: True
Entry: type=market, start=mid, interval=3s, steps=4


## Connect to Alpaca & Initialize Components

In [3]:
# Alpaca + Data Layer
alpaca = AlpacaClientManager(paper=config.paper_trading)
vix_fetcher = VixDataFetcher()
greeks_calc = GreeksCalculator()
data_manager = DataManager(alpaca, config)

print(f"Alpaca connected (paper={alpaca.paper})")
print(f"VIX: {vix_fetcher.get_current_vix():.2f}")

Alpaca connected (paper=True)
VIX: 19.62


In [4]:
# Scanner + Execution + Risk + Metadata
scanner = StrategyScanner(
    config=config,
    equity_fetcher=data_manager.equity_fetcher,
    options_fetcher=data_manager.options_fetcher,
    greeks_calc=greeks_calc,
)
execution = ExecutionEngine(alpaca, config)
risk_manager = RiskManager(config)
metadata = StrategyMetadataStore(path="strategy_metadata.json")

print("Scanner, Execution, Risk, Metadata — OK")

Scanner, Execution, Risk, Metadata — OK


In [5]:
# Build TradingLoop
loop = TradingLoop(
    config=config,
    data_manager=data_manager,
    scanner=scanner,
    metadata_store=metadata,
    risk_manager=risk_manager,
    execution=execution,
    vix_fetcher=vix_fetcher,
    greeks_calc=greeks_calc,
    alpaca_manager=alpaca,
)
print("TradingLoop ready")

  Daily log: logs/2026-02-18.json
TradingLoop ready


## Portfolio Status

In [6]:
print_portfolio_status(alpaca)
#toggle: account information, [Collat, Avail Cash], current positions, Open Orders, Recent Orders .head(10), what is time here? EST?


Alpaca Account Information
Account status:              AccountStatus.ACTIVE
Cash available:              $1,007,551.75
Buying power (with margin):  $4,010,142.30
Portfolio value:             $1,005,871.75
Options trading level:       3
Trading blocked:             False

Current Positions (3):
  Symbol                    Qty Side       Strike        Entry        Current      Mkt Value     Unreal P/L   Collateral
  ------------------------------------------------------------------------------------------------------------------------
  NEM260220P00110000         -5 short  $   110.00 $       0.61 $         0.35 $      -175.00 $       130.00 $  55,000.00
  SNDK260220P00525000        -1 short  $   525.00 $       2.67 $         1.10 $      -110.00 $       157.50 $  52,500.00
  WDC260227P00260000         -3 short  $   260.00 $       4.25 $         4.65 $    -1,395.00 $      -120.00 $  78,000.00

  Total collateral tied up: $185,500.00
  Available cash (after collateral): $822,051.75

Open O

## Scan Diagnostics

In [7]:
print_scan_diagnostics(scanner, config, alpaca)

# Portfolio not needed. Better header for passed equity filter. Toggle for summary, passed equity, passed options,  

Universe Scan
Alpaca cash:                               $1,007,551.75
Short position collateral:                 $185,500.00
Available capital (cash - collateral):      $822,051.75
Max position size (10.0%):                $82,205.18

Symbols scanned:                         501
Passed equity filter:                     27
Passed equity + options filter:            10

Passed equity filter (27):
  Symbol       Price      SMA8     SMA20     SMA50      BB20    RSI   Collateral  Opts
  ----------------------------------------------------------------------------------------
  APH      $  147.65    144.41    146.23    141.88    154.73   51.5 $    14,765     0
  AWK      $  131.21    127.82    128.18    129.68    131.44   57.9 $    13,121     0
  CCI      $   87.44     86.53     85.94     87.27     88.87   54.0 $     8,744     0
  CNC      $   43.07     40.07     42.00     42.06     44.47   54.0 $     4,307     2
  COR      $  360.15    360.06    355.96    347.96    363.71   54.2 $    36,01

## Run Strategy

In [12]:
# Run a single cycle to verify everything works
summary = await loop.run_cycle()
print(f"\nCycle summary: Exits={summary.get('exits', 0)}, Entries={summary.get('entries', 0)}")

# Print only 
#   Short position collateral:      $  182,500.00
#   Available capital:               $  824,876.85
#   Max position size (10.0%):     $   82,487.68



  Alpaca cash:                    $1,007,376.85
  Short position collateral:      $  182,500.00
  Available capital:               $  824,876.85
  Max position size (10.0%):     $   82,487.68
  Active positions:                           3

Cycle summary: Exits=0, Entries=0


In [13]:
# Live Trading Loop
poll_interval, max_cycles = (60, 100)

if execution.paper:
    print("Paper trading mode confirmed")
else:
    print("WARNING: LIVE TRADING MODE")
    print("Switch to paper trading before running the loop!")

await loop.run(poll_interval, max_cycles)

Paper trading mode confirmed

CSP TRADING LOOP STARTED
Poll Interval: 60s
Paper Trading: True


--- Cycle 1 @ 2026-02-17 18:48:53 EST ---

  Alpaca cash:                    $1,007,376.85
  Short position collateral:      $  182,500.00
  Available capital:               $  824,876.85
  Max position size (10.0%):     $   82,487.68
  Active positions:                           3
  Market closed — exiting loop.

Trading loop ended.
Total cycles: 1
Final portfolio: {'active_positions': 3, 'total_collateral': 182500.0, 'symbols': ['STX', 'SNDK', 'NEM']}


## Utilities

In [9]:
# Uncomment to liquidate all holdings:
liquidate_all_holdings(alpaca)

LIQUIDATING ALL HOLDINGS

Open Positions (4):
  NEM260220P00110000   qty=    -5 side=short  strike=$ 110.00 exp=2026-02-20 price=$  0.73
  SNDK260220P00525000  qty=    -1 side=short  strike=$ 525.00 exp=2026-02-20 price=$  3.40
  STX260227P00375000   qty=    -2 side=short  strike=$ 375.00 exp=2026-02-27 price=$  7.30
  WDC260227P00247500   qty=    -4 side=short  strike=$ 247.50 exp=2026-02-27 price=$  5.10
  Total collateral: $281,500.00

Cancelling all open orders...

Closing active positions...
  Closing NEM260220P00110000 qty=5 order_id=bcd35cc6-f436-48b8-87b2-1e0768f0eaf0
  Closing SNDK260220P00525000 qty=1 order_id=f7846a7e-f180-4ed7-a3cd-413585980a62
  Closing STX260227P00375000 qty=2 order_id=4977b255-10fc-4145-bee0-2c68939053f8
  Closing WDC260227P00247500 qty=4 order_id=8b352e2a-3889-4df5-b354-8977bccea0f2

All positions closed.

Final: Cash=$1,005,657.01 | Buying power=$4,029,482.56
